In [ ]:
import os

os.environ["GRAPHISTRY_USERNAME"] = input("Please enter your graphistry username")
os.environ["GRAPHISTRY_PASSWORD"] = input("Please enter your graphistry password")
os.environ["OPENAI_API_KEY"] = input("Please enter your OpenAI API key")

In [ ]:
import os
import pathlib
import cognee
from cognee.infrastructure.databases.relational import create_db_and_tables

notebook_path = os.path.abspath("")
data_directory_path = str(
    pathlib.Path(os.path.join(notebook_path, ".data_storage/code_graph")).resolve()
)
cognee.config.data_root_directory(data_directory_path)
cognee_directory_path = str(
    pathlib.Path(os.path.join(notebook_path, ".cognee_system/code_graph")).resolve()
)
cognee.config.system_root_directory(cognee_directory_path)

await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)

await create_db_and_tables()

In [ ]:
from os import path
from pathlib import Path
from cognee.infrastructure.files.storage import LocalStorage
import git

notebook_path = path.abspath("")
repo_clone_location = path.join(notebook_path, ".data/graphrag")

LocalStorage.remove_all(repo_clone_location)

git.Repo.clone_from(
    "git@github.com:microsoft/graphrag.git",
    Path(repo_clone_location),
    branch="main",
    single_branch=True,
)

In [ ]:
from cognee.tasks.repo_processor import (
    get_repo_file_dependencies,
)
from cognee.tasks.storage import add_data_points
from cognee.modules.pipelines.tasks.Task import Task

detailed_extraction = True

tasks = [
    Task(get_repo_file_dependencies, detailed_extraction=detailed_extraction),
    Task(add_data_points, task_config={"batch_size": 100 if detailed_extraction else 500}),
]

In [ ]:
from cognee.modules.pipelines import run_tasks
from uuid import uuid5, NAMESPACE_OID

pipeline = run_tasks(tasks, uuid5(NAMESPACE_OID, repo_clone_location), repo_clone_location, "code_graph_pipeline")

async for result in pipeline:
    print(result)

In [ ]:
from cognee.shared.utils import render_graph

await render_graph()

# Let's check the evaluations

In [ ]:
from cognee import search, SearchType

results = await search(query_type=SearchType.CODE, query_text="def create_graphrag_config")

print(results)
